In [1]:
import sys
!{sys.executable} -m pip install -r requirements.txt

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.7/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import tinvest
import pandas as pd
import ipywidgets as widgets

from IPython.display import clear_output
from datetime import datetime, timedelta, timezone

# Get token

In [3]:
f = open("token.txt", "r")

TOKEN = f.read()
TOKEN

't.IodbD3G5wmc4_vOlZDLdTKe4N9VVpCoEIAmbTcFhqWVACCiXMTKCLYm0GP7v2SRf-ozJb5Vq6Ix67Bv5-ydMDA'

# Get companies ranks

In [4]:
companies_pd = pd.read_parquet("data/companies_rank.parquet")
companies_pd.head()

,Rank,Company,Country/Territory,Sales,Profits,Assets,Market Value
1,1.0,ICBC,China,$177.2 B,$45.3 B,"$4,322.5 B",$242.3 B
2,2.0,China Construction Bank,China,$162.1 B,$38.9 B,"$3,822 B",$203.8 B
3,3.0,JPMorgan Chase,United States,$142.9 B,$30 B,"$3,139.4 B",$291.7 B
4,4.0,Berkshire Hathaway,United States,$254.6 B,$81.4 B,$817.7 B,$455.4 B
5,5.0,Agricultural Bank of China,China,$148.7 B,$30.9 B,"$3,697.5 B",$147.2 B


# Get markets

In [5]:
client = tinvest.SyncClient(TOKEN, use_sandbox=True)
api = tinvest.MarketApi(client)

In [6]:
# TODO Add concurencies

def markets_list_2_df(markets_instruments_list):
    markets_instruments_data = [ {
            'currency': markets_instrument.currency,
            'figi': markets_instrument.figi,
            'isin': markets_instrument.isin,
            'lot': markets_instrument.lot,
            'min_price_increment': markets_instrument.min_price_increment,
            'name': markets_instrument.name,
            'ticker': markets_instrument.ticker,
            'type': markets_instrument.type
        }
        for markets_instrument
        in markets_instruments_list]

    return pd.DataFrame(markets_instruments_data)

markets_instruments_df = pd.DataFrame()

# Get Stocks
response = api.market_stocks_get()
if response.status_code == 200:
    # Parse
    markets_instruments_df = pd.concat([markets_instruments_df,
                                        markets_list_2_df(response.parse_json().payload.instruments)])
else:
    print("ERROR:", response.parse_error())

# Get Bonds
response = api.market_bonds_get()
if response.status_code == 200:
    # Parse
    markets_instruments_df = pd.concat([markets_instruments_df,
                                        markets_list_2_df(response.parse_json().payload.instruments)])
else:
    print("ERROR:", response.parse_error())
    
# Get ETFs
response = api.market_etfs_get()
if response.status_code == 200:
    # Parse
    markets_instruments_df = pd.concat([markets_instruments_df,
                                        markets_list_2_df(response.parse_json().payload.instruments)])
else:
    print("ERROR:", response.parse_error())

In [7]:
markets_instruments_df.shape

(1794, 8)

In [8]:
markets_instruments_df.head()

,currency,figi,isin,lot,min_price_increment,name,ticker,type
0,Currency.usd,BBG000HLJ7M4,US45867G1013,1,0.01,InterDigItal Inc,IDCC,InstrumentType.stock
1,Currency.usd,BBG002293PJ4,US74967X1037,1,0.01,RH,RH,InstrumentType.stock
2,Currency.usd,BBG000CTQBF3,US8552441094,1,0.01,Starbucks Corporation,SBUX,InstrumentType.stock
3,Currency.usd,BBG000BPL8G3,US5537771033,1,0.01,MTS Systems Corp,MTSC,InstrumentType.stock
4,Currency.usd,BBG000BH0FR6,US8125781026,1,0.01,Seattle Genetics Inc,SGEN,InstrumentType.stock


# Merge Forbes rank data with Tinkoff instruments

In [10]:
top_markets_instruments_df = pd.merge(markets_instruments_df, companies_pd, how='inner', left_on='name', right_on='Company')
top_markets_instruments_df.head()

,currency,figi,isin,lot,min_price_increment,name,ticker,type,Rank,Company,Country/Territory,Sales,Profits,Assets,Market Value
0,Currency.usd,BBG000BF0K17,US1491231015,1,0.01,Caterpillar,CAT,InstrumentType.stock,132.0,Caterpillar,United States,$51 B,$5.3 B,$75.9 B,$63 B
1,Currency.usd,BBG000QW7VC1,US40412C1018,1,0.01,HCA Healthcare,HCA,InstrumentType.stock,225.0,HCA Healthcare,United States,$51.7 B,$3 B,$45.4 B,$37.2 B
2,Currency.usd,BBG005YHY0Q7,US47215P1066,1,0.01,JD.com,JD,InstrumentType.stock,238.0,JD.com,China,$83.4 B,$1.8 B,$37.3 B,$53.6 B
3,Currency.usd,BBG000L9CV04,US9113121068,1,0.01,United Parcel Service,UPS,InstrumentType.stock,143.0,United Parcel Service,United States,$74.8 B,$4.3 B,$57.9 B,$81.3 B
4,Currency.usd,BBG000BB6KF5,US59156R1086,1,0.01,MetLife,MET,InstrumentType.stock,80.0,MetLife,United States,$69 B,$5.9 B,$746.3 B,$32.8 B


# Get Candles

In [11]:
figi = markets_instruments_df.figi.iloc[0]
datetime_to = (datetime.now(timezone.utc) - timedelta(days=2)).isoformat()
datetime_from = (datetime.now(timezone.utc) - timedelta(days=3)).isoformat()

print(figi)
print(datetime_from)
print(datetime_to)

response = api.market_candles_get(figi,
                                 datetime_from,
                                  datetime_to,
                                 tinvest.CandleResolution.min15)
if response.status_code == 200:
    # Parse
    print(response.parse_json())
else:
    print("ERROR:", response.parse_error())

BBG000HLJ7M4
2020-06-18T16:12:51.540217+00:00
2020-06-19T16:12:51.540149+00:00
payload=Candles(candles=[Candle(c=54.37, figi='BBG000HLJ7M4', h=54.54, interval=<CandleResolution.min15: '15min'>, l=54.31, o=54.51, time='2020-06-18T16:15:00Z', v=1656), Candle(c=54.23, figi='BBG000HLJ7M4', h=54.4, interval=<CandleResolution.min15: '15min'>, l=54.17, o=54.36, time='2020-06-18T16:30:00Z', v=1544), Candle(c=54.37, figi='BBG000HLJ7M4', h=54.39, interval=<CandleResolution.min15: '15min'>, l=54.25, o=54.25, time='2020-06-18T16:45:00Z', v=1519), Candle(c=54.51, figi='BBG000HLJ7M4', h=54.51, interval=<CandleResolution.min15: '15min'>, l=54.32, o=54.4, time='2020-06-18T17:00:00Z', v=1633), Candle(c=54.42, figi='BBG000HLJ7M4', h=54.47, interval=<CandleResolution.min15: '15min'>, l=54.4, o=54.41, time='2020-06-18T17:15:00Z', v=2666), Candle(c=54.59, figi='BBG000HLJ7M4', h=54.59, interval=<CandleResolution.min15: '15min'>, l=54.47, o=54.47, time='2020-06-18T17:30:00Z', v=2065), Candle(c=54.63, figi='B